# Integrated Bioinformatics Project : Flux_Balance_F_Prausnitzii 
#### Students: Michael Shawn Neilsson, Biancamaria FLorenzi, Prabhat Juyal, Tim Blokker
#### Supervisors: Clémence Joseph, Karoline Faust

In [ ]:
#import packages
from libchebipy import ChebiEntity
import molmass
import io
import cobra
import functions
import re
from functions import *
import cobrababel
import json
import requests
import pandas as pd

In [ ]:
#model = cobra.io.read_sbml_model('Data/f_prausnitzii.xml')
#info(model)
model=cobra.io.load_matlab_model('Data/iFpraus_v_1_0.mat')
info(model)

In [ ]:
x=[model.reactions.get_by_id(compound_id).name for compound_id in model.medium]
x=[compound[:(len(compound)-3)] for compound in x]
x=[compoundname.replace("EX_", "") for compoundname in x]
x=[compoundname.replace("excha", "").strip() for compoundname in x]
";".join(x)

In [ ]:
url = "http://api.xialab.ca/mapcompounds"

payload = "{\n\t\"queryList\": \"%s;\",\n\t\"inputType\": \"name\"\n}" %";".join(x)
headers = {
    'Content-Type': "application/json",
    'cache-control': "no-cache",
    }

response = requests.request("POST", url, data=payload, headers=headers)

In [ ]:
dict=json.loads(response.text)
mapped_compounds=pd.DataFrame(dict)
mapped_compounds.head()

In [ ]:
mol_weight=[]
for i in mapped_compounds["chebi_id"]:
    try:
        x= ChebiEntity(str(i))
        mol_weight.append(x.get_mass())
    except:
        mol_weight.append("NA")
        continue

In [ ]:
mapped_compounds["mol_weight"]=mol_weight
mapped_compounds["BiGG_ID"]=[model.reactions.get_by_id(compound_id).id[3:] for compound_id in model.medium]
weights=mapped_compounds[["query", "BiGG_ID", "hit", "mol_weight"]]
weights.head()

In [ ]:
#outputmodel= cobra.io.save_json_model(model, "Data/iFpraus.json")

In [ ]:
#unaltered model
model1=model.copy()
print("\nModel Medium")
model1.objective = {model1.reactions.get_by_id('Biomass_FP'): 1}
print(model1.summary())

### What are the different reactions
e: extracellular

c: cytosol

EX_ : exchange reaction
### Gifu Anaerobic Medium (mGAM) from:
https://hyserve.com/files/05433_GAM-Broth_Modified_final.pdf


- Peptone, Soya Peptone, Proteose Peptone: source of amino acids -> most likely all of them 
- Yeast extract, liver extract, meat extract, digested serum -> 
    - amino acids, 
    - '4abz(e)' 
    - '5oxpro(e)'
    - thm(e)' thiamin vitamin b1
    - biotin -> btn |from yeast extract? 
    - 'rbflvrd(e)', 'ribflv(e)' vitamin b2
    - 'pydam(e)' pydx pydxn vitamin b6 from yeast extract
    - 'pnto_R(e)' pantothenic acid from yeast extract?
    - 'ptrc(e)' probably from meat extract(e)
    - 'spmd(e)' from meat extract and linked to ptrc
    - cbl1   Cyanocobalamin is a cobalt-containing coordination compound and a natural water-soluble vitamin of the B-complex family 
    - adocbl
    - folate fol is of B vitamin family 
    - tautin from meat extract
    - nac is nicotinate and also vitamin b and so is nacm nicotinamide
    - hxan, gua , ins are all some kind of nucleic acid that is probably present in yeast extract
    - trace elements: Ca2? Cu2? cobalt2?  mg2? pi(e)? so4?
- Dextrose -> same as glucose (https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:17634) and so -> glc_D
- Soluble starch -> starch1 (potatoe starch)
- L-Tryptophane -> trp__L
- L-Cystein Hydrochloride -> cys_L and cl and h (hydrochloride<-> HCL becomes cl and h)
- Sodium Thioglycolate -> thiog and na1
- L-Arginine -> arg_L
- Vitamine K1 -> phllqne https://pubchem.ncbi.nlm.nih.gov/compound/Vitamin-K1#section=Depositor-Supplied-Synonyms 
- Hemin -> Iron and Chlorine  fe3 and cl  and perhaps ppp9 https://pubchem.ncbi.nlm.nih.gov/#query=3-%5B18-(2-carboxyethyl)-8%2C13-bis(ethenyl)-3%2C7%2C12%2C17-tetramethylporphyrin-21%2C23-diid-2-yl%5Dpropanoic%20acid%3Biron(3%2B)%3Bchloride 
- Potassium Dihydrogen Phosphate -> pi, k, h
- Sodium Chloride -> na1 cl

Digested serum : https://www.nebiolabs.com.au/-/media/catalog/datacards-or-manuals/p8108datasheet-lot0021306.pdf	

Generally useful page ; https://pubchem.ncbi.nlm.nih.gov/#query=Hemin

check also: https://opencobra.github.io/cobratoolbox/latest/tutorials/tutorialMetabotoolsI.html 

In [ ]:
import numpy as np
aa=("ala,arg,asn,asp,asx,cys,glu,gln,glx,gly,his,ile,leu,lys,met,phe,pro,ser,thr,trp,tyr,val").split(",")
aa=[aminoacid +"_L(e)" for aminoacid in aa]

In [ ]:
import re
r=re.compile(r"EX_.*?\)")
ex_rxn=re.findall(r, str(model.exchanges))
MediumMgam_complete=[x[3:len(x)] for x in ex_rxn]
print(MediumMgam_complete)

In [ ]:
MediumMgam=['4abz(e)','5oxpro(e)','Lcyst(e)', 'Lcystin(e)', 'ade(e)', 'adocbl(e)','ala_D(e)', 'ala_L(e)', 'arg_L(e)', 'asn_L(e)', 'asp_L(e)', 'btn(e)', 'ca2(e)', 'cbl1(e)', 'cbl2(e)', 'cl(e)', 'cobalt2(e)', 'cu2(e)', 'cys_L(e)', 'fe2(e)', 'fe3(e)', 'fol(e)', 'glc(e)', 'gln_L(e)', 'glu_L(e)', 'gly(e)',  'gthox(e)', 'gthrd(e)', 'gua(e)', 'h(e)', 'h2o(e)', 'his_L(e)', 'hxan(e)', 'ile_L(e)', 'ins(e)',  'k(e)',  'leu_L(e)', 'lys_L(e)', 'met_D(e)', 'met_L(e)', 'mg2(e)', 'na1(e)', 'nac(e)', 'ncam(e)',  'phe_L(e)', 'pi(e)', 'pnto_R(e)', 'pro_L(e)', 'ptrc(e)', 'pydam(e)', 'pydx(e)', 'pydxn(e)', 'rbflvrd(e)', 'ribflv(e)', 'ser_L(e)', 'so4(e)', 'spmd(e)', 'strch1(e)',  'taur(e)', 'thm(e)', 'thr_L(e)', 'thymd(e)', 'trp_L(e)', 'tyr_L(e)', 'ura(e)', 'val_L(e)', 'xan(e)', 'xyl_D(e)']

In [99]:
Weights_Mgam=weights[weights["BiGG_ID"].isin(MediumMgam)]
Weights_Mgam

,query,BiGG_ID,hit,mol_weight
1,4 Aminobenzoate,4abz(e),,NA
2,5oxpro,5oxpro(e),,NA
3,L-cysteate,Lcyst(e),Cysteic acid,169.156
4,L-Cystine,Lcystin(e),L-Cystine,240.303
12,Adenine,ade(e),Adenine,135.127
13,Adenosylcobalamin,adocbl(e),Adenosylcobalamin,1579.58
14,D-Alanine,ala_D(e),D-Alanine,89.0932
15,L-Alanine,ala_L(e),L-Alanine,89.0932
25,L-Arginine,arg_L(e),L-Arginine,174.201
26,L-Asparagine,asn_L(e),L-Asparagine,132.118


In [ ]:
model_mgam=medium(model.copy(),MediumMgam)
print('\nMedium_mgam')
for i in MediumMgam:
    #if model_mgam.exchanges.get_by_id(i).lower_bound != 0:
     #model_mgam.exchanges.get_by_id(i).lower_bound=-1000
    model_mgam.exchanges.get_by_id('EX_glc(e)').lower_bound=-2.7
model_mgam.summary()

In [ ]:
for i in MediumMgam:
    print(model_mgam.reactions.get_by_id(i))
    print(model_mgam.reactions.get_by_id(i).lower_bound)

In [ ]:
find_match(model_mgam,MediumMgam)

In [ ]:
for i in find_match(MediumMgam,)[1]:
    r = re.compile(r'{0}.+?{1}?.+?{2}?.*?'.format(i[0:5],i[5], i[6]))
    newlist = list(filter(r.findall, x)) # Read Note
    print(newlist)

In [ ]:
fba = model.optimize()
fluxes = deleteNull(fba.fluxes)
print('\nActive reaction : %i which represent %f%% of the model' %(len(fluxes),len(fluxes)/len(model.reactions)*100))
print(fba.objective_value)

In [ ]:
from cobra.medium import minimal_medium

max_growth = model.slim_optimize()
minimal_medium(model, 0.005)

In [ ]:
from cobra.medium import minimal_medium
max_growth = model_mgam.slim_optimize()
minimal_medium(model_mgam, max_growth)

---
## Reinforced Clostridial Medium (RCM)
http://www.oxoid.com/UK/blue/prod_detail/prod_detail.asp?pr=CM0149&org=53&c=UK&lang=EN

* Yeast Extract -> [lots of stuff in it, unsure of what's garbage](https://www.chemicalbook.com/ChemicalProductProperty_EN_CB9440339.htm#:~:text=Yeast%20extract%20has%20a%20protein,aromatic%20compounds%20and%20other%20components.)
    * Glutathione: gthrd (reduced version)
    * 18 AA's: see above
    * Dextran: glc__D (it's made of glucose)
    * Mannan: mannan
    * Trehalose: tre
    * B-vitamins: (Clemence mentioned vitamins being important so I'm putting them all in)
        * \1. Thiamin -> thm
        * \2. Riboflavin -> ribflv
        * \3. Niacin -> trp__L (made of tryptophan)
        * \5. Pantothenic Acid -> pnto__R
        * \6. Pyridoxine -> pydxn
        * \7. Biotin -> btn
        * \9. Folic Acid -> ... can't find anything
        * \12. Cobalamin -> b12 (or cbl1)
    * Biotin
* Peptone -> Sticking with the above logic, probably a source of all AAs
* Glucose -> Assuming D_Glucose: glc__D
* Soluble Starch -> starch1200
* Sodium Chloride -> na1, cl
* Sodium Acetate -> Assuming breaking down into sodium and acetate: ac, na1
* Cysteine Hydrochloride -> cys_L, cl, h (could also be cys_D? cys_L is more widely used)
* Agar: Agarose (70%) and Agaropectin (30%) [according to Wiki](https://en.wikipedia.org/wiki/Agar)
    * Agarose: D-galactose & 3,6-anhydro-L-galactopyranose
        * D-galactose: gal
        * 3,6-anhydro-L-galactopyranose: [similar structure & compositon to beta-D-allose](https://pubchem.ncbi.nlm.nih.gov/#query=CID67020466%20structure&tab=similarity)
            * D-Allose: all__D
    * Agaropectin: D-galactose, L-galactose, pyruvate, sulfate
        * D-galactose: gal
        * L-galactose: gal__L
        * Pyruvate: pyr
        * Sulfate: so4

| Component              | Concentration (g/L) | BiGG Metabolites                                                                          |
|------------------------|---------------------|-------------------------------------------------------------------------------------------|
| Yeast Extract          | 13.0                | (see all AA's), gthrd, glc\__D, mannan, tre, thm, ribflv, trp\__L, pnto\__R, pydxn, btn, b12 |
| Peptone                | 10.0                | (see all AA's)                                                                            |
| Glucose                | 5.0                 | glc_D                                                                                     |
| Soluble Starch         | 1.0                 | starch1200                                                                                |
| Sodium Chloride        | 5.0                 | na1, cl                                                                                   |
| Sodium Acetate         | 3.0                 | na1, ac                                                                                   |
| Cysteine Hydrochloride | 0.5                 | cys_L, cl, h                                                                              |
| Agar                   | 0.5                 | gal, all\__D, gal\__L, pyr, so4                                                           |

In [ ]:
medium_rcm = list(('gthrd','glc__D','mannan','tre','thm','ribflv','trp__L','pnto__R','pydxn','btn','b12',
             'starch1200','na1','ac','cl','cys_L','gal','all__D','gal__L','pyr','so4'))
medium_rcm=[mets+"(e)" for mets in medium_rcm]
model_rcm=medium(model.copy(), medium_rcm)
for i in range(1,len(model_rcm.reactions)):
    model_rcm.reactions[i].upper_bound=1000
    model_rcm.reactions[i].lower_bound=-1000
model_rcm.objective = {model_rcm.reactions.get_by_id('Biomass_FP'): 1}
model_rcm.summary()

In [ ]:
model_mmcb.reactions.get_by_id('mannan')

In [ ]:
# Post-optimization
fba_rcm = model_rcm.optimize()
fluxes = deleteNull(fba_rcm.fluxes)
print('\nActive reaction : %i which represent %f%% of the model' %(len(fluxes),len(fluxes)/len(model.reactions)*100))
print(model_rcm.objective.value)

---
## mMCB

(There are also supplements mentioned in the paper but this is the base medium)
* [Bacteriological Peptome (6.5) (Oxoid)](http://www.oxoid.com/UK/blue/prod_detail/prod_detail.asp?pr=LP0037&c=UK&lang=EN): 
    * Vague, decided to go with polypeptides: <b>polypep</b>
    * Nitrogen: <b>n2</b>
* [Soy Peptome (5.0) (Oxoid)](http://www.oxoid.com/UK/blue/prod_detail/prod_detail.asp?pr=LP0044&cat=&c=UK&lang=EN) --> another vague one:
    * Stachyose: <b>stys</b>
    * Raffinose: <b>raffin</b>
    * Sucrose: <b>sucr</b>
    * Nitrogen: <b>n2</b>
* [Yeast Extract (3.0) (VWR International, Darmstadt, Germany)](https://us.vwr.com/store/product/7437401/vwr-life-science-yeast-extract-bacteriological-grade):
    * Vitamin B (same logic as RCM):
        * 1. Thiamin -> <b>thm</b>
        * 2. Riboflavin -> <b>ribflv</b>
        * 3. Niacin -> <b>trp__L</b> (made of tryptophan)
        * 5. Pantothenic Acid -> <b>pnto__R</b>
        * 6. Pyridoxine -> <b>pydxn</b>
        * 7. Biotin -> <b>btn</b>
        * 9. Folic Acid -> ... can't find anything
        * 12. Cobalamin -> <b>b12</b> (or cbl1)
* [Tryptone (2.5) (Oxoid)](http://www.oxoid.com/UK/blue/prod_detail/prod_detail.asp?pr=LP0042&c=UK&lang=EN): 
    * Tryptophan: <b>trp__L</b>
* NaCL (1.5) (VWR International, Darmstadt, Germany):
    * NaCl: <b>na1, cl</b>
* K<sub>2</sub>HPO<sub>4</sub> (1.0) (Merck International, Darmstadt, Germany):
    * Potassium: <b>k</b>
    * Phosphate: <b>p1</b>
    * Hydrogen: <b>h2</b>
* KH<sub>2</sub>PO<sub>4</sub> (1.0) (Merck International, Darmstadt, Germany):
    * Potassium: <b>k</b>
    * Phosphate: <b>p1</b>
    * Hydrogen: <b>h2</b>
* Na<sub>2</sub>SO<sub>4</sub> (2.0) (VWR):
    * Sodium: <b>na1</b>
    * Sulfate: <b>so4</b>
* MgSO<sub>4</sub>*7H<sub>2</sub>O (1.0) (Merck):
    * Magnesium: <b>mg2</b>
    * Sulfate: <b>so4</b>
* CaCl<sub>2</sub>*2H<sub>2</sub>O (0.1) (Merck):
    * Calcium Chloride: <b>ca2, cl</b>
* NH<sub>4</sub>Cl (1.0) (Merck):
    * Ammonium Chloride: <b>nh4, cl</b>
* Cysteine-HCL (0.4) (Merck):
    * Cysteine: <b>cys__L</b>
    * HCL: <b>h2, cl</b>
* NaHCO<sub>3</sub> (0.2) (VWR):
    * Sodium: <b>na1</b>
    * Bicarbonate: <b>hco3<b>
* MnSO<sub>4</sub>*H<sub>2</sub>O (0.05) (VWR):
    * Manganese: <b>mn2</b>
    * Sulfate: <b>so4</b>
* FeSO<sub>4</sub>*7H<sub>2</sub>O (0.005) (Merck):
    * Iron: <b>fe</b>
    * Sulfate: <b>so4</b>
* ZnSO<sub>4</sub>*7H<sub>2</sub>O (0.005) (VWR):
    * Zinc: <b>zn2</b>
    * Sulfate: <b>so4</b>
* Hemin (0.005) (Sigma-Aldrich, Steinheim, Germany):
    * ... contains iron
* Menadione (0.005) (S-A):
    * <b>mndn</b>
* Resazurin (0.001) (S-A):
    * Fluoro identifier



| Component                            | Concentration (g/L) | BiGG Metabolites                                |
|--------------------------------------|---------------------|-------------------------------------------------|
| Bacteriological Peptome              | 6.5                 | polypep, n2                                     |
| Soy Peptome                          | 5.0                 | stys, raffin, sucr, n2                          |
| Yeast Extract                        | 3.0                 | thm, ribflv, trp\__L, pnto\__R, pydxn, btn, b12 |
| Tryptone                             | 2.5                 | trp\__L                                         |
| NaCL                                 | 1.5                 | na1, cl                                         |
| K<sub>2</sub>PO<sub>4</sub>          | 1.0                 | k, p1, h2                                       |
| KH<sub>2</sub>PO<sub>4</sub>         | 1.0                 | k, p1, h2                                       |
| Na<sub>2</sub>SO<sub>4</sub>         | 2.0                 | na1, so4                                        |
| Mg<sub>2</sub>SO<sub>4</sub>         | 1.0                 | mg2, so4                                        |
| CaCl<sub>2</sub>*2H<sub>2</sub>O     | 0.1                 | ca2, cl                                         |
| NH<sub>4</sub>Cl                     | 1.0                 | nh4, cl                                         |
| Cysteine-HCL                         | 0.4                 | cys__L, h2, cl                                  |
| NaHCO<sub>3</sub>                    | 0.2                 | na1, hco3                                       |
| MnSO<sub>4</sub>*H<sub>2</sub>O      | 0.05                | mn2, so4                                        |
| FeSOMnSO<sub>4</sub>*7H<sub>2</sub>O | 0.005               | fe, so4                                         |
| ZnSO<sub>4</sub>*7H<sub>2</sub>O     | 0.005               | zn2, so4                                        |
| Hemin                                | 0.005               |                                                 |
| Menadione                            | 0.005               | mndn                                            |
| Resazurin                            | 0.001               |                                                 |

In [ ]:
model.copy()

In [ ]:
medium_mmcb = list(('polypep','n2','stys','raffin','sucr','n2','thm','ribflv','trp__L',
                    'pnto__R','pydxn','btn','b12','trp__L','na1','cl','k','p1','h2','na1',
                    'so4','mg2','ca2','nh4','cys__L','h2','cl','na1','hco3','mn2','zn2','mndnb'))

In [ ]:
medium_mmcb=[mets+"(e)" for mets in medium_mmcb]
model_mmcb=medium(model.copy(), medium_mmcb)
unused = []
for i in range(1,len(model_mmcb.reactions)):
    model_mmcb.reactions[i].upper_bound=1000
    if model_mmcb.reactions[i].lower_bound != 0:
        model_mmcb.reactions[i].lower_bound=-10
    else:    
        model_mmcb.reactions[i].lower_bound=-
model_mmcb.objective = {model_mmcb.reactions.get_by_id('Biomass_FP'): 1}
model_mmcb.summary()

# Find list of all metabolites not in media
# Express in mmol/g/L
# Set as -'ve flux lower bound

In [ ]:
# Post-optimization
fba_mmcb = model_mmcb.optimize()
fluxes_mmcb = deleteNull(fba_mmcb.fluxes)
print('\nActive reaction : %i which represent %f%% of the model' %(len(fluxes_mmcb),len(fluxes_mmcb)/len(model.reactions)*100))
print(model_mmcb.objective.value)